In [1]:
! pip install --no-index --find-links /kaggle/input/pycocotools/wheelhouse pycocotools

Looking in links: /kaggle/input/pycocotools/wheelhouse
Processing /kaggle/input/pycocotools/wheelhouse/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl


In [2]:
!pip install /kaggle/input/hubmap-model/ultralytics-8.0.119-py3-none-any.whl

Processing /kaggle/input/hubmap-model/ultralytics-8.0.119-py3-none-any.whl


In [3]:
from ultralytics import YOLO
from PIL import Image
import cv2
from pathlib import Path
import pandas as pd

In [4]:
import base64
import numpy as np
from pycocotools import _mask as coco_mask
import typing as t
import zlib


def encode_binary_mask(mask: np.ndarray) -> t.Text:
    """Converts a binary mask into OID challenge encoding ascii text."""

    # check input mask --
    if mask.dtype != np.bool_:
        raise ValueError(
            "encode_binary_mask expects a binary mask, received dtype == %s" %
            mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask expects a 2d mask, received shape == %s" %
            mask.shape)

  # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str

In [5]:
ROOT = Path("/kaggle/input/hubmap-hacking-the-human-vasculature")

image_folder = "test"

image_list = [str(i) for i  in (ROOT / image_folder).glob('*.tif')]
model = YOLO("/kaggle/input/hubmap-model/best.pt")

In [6]:
df = pd.DataFrame(columns=['id' ,'height' ,'width', 'prediction_string'])

# cnt = 0
for i in (ROOT / image_folder).glob('*.tif'):
    im2 = cv2.imread(str(i))
    results = model.predict(source=im2)  # save predictions as labels
    pred_string = ""
    
    if results[0].masks != None:
    
        masks = results[0].masks.data.to('cpu').detach().numpy().astype(bool)
        boxes = results[0].boxes.xyxy.to('cpu').detach().numpy()
        score = results[0].boxes.conf.to('cpu').detach().numpy()

        

        for idx in range(len(masks)):
            if score[idx] > 0.5:
                binary_mask = encode_binary_mask(masks[idx]).decode("utf-8")
                pred_string += f"0 {score[idx]} {binary_mask} "

    new_line = pd.Series({'id': i.stem, 'height': 512 , 'width': 512 , 'prediction_string':pred_string})
    df = pd.concat([df, new_line.to_frame().T], ignore_index=True)
    
#     cnt += 1
    
#     if cnt == 100:
#         break
    
df.to_csv("submission.csv" , index=False)
print(df)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

             id height width  \
0  72e40acccadf    512   512   

                                   prediction_string  
0  0 0.9163355827331543 eNqLjggytUyxN/Mx8jP0B2FDA...  
